<a href="https://colab.research.google.com/github/Tonge-Shim/1/blob/master/xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from pandas import DataFrame
warnings.filterwarnings(action='ignore')
%matplotlib inline

In [2]:
df= pd.read_csv('samp_cst_feat.csv')
dtype = pd.read_excel('variable_dtype.xlsx')
dtype_categ = dtype[(dtype['dType']=='categorical')]
dtype_numrc = dtype[(dtype['dType']=='numerical')]
numrc_var = dtype_numrc['Variable_Name']
categ_var = dtype_categ['Variable_Name']
numrc_var = numrc_var.values.tolist()
categ_var = categ_var.values.tolist()#0,1
x=df.drop(categ_var, axis=1).drop(['cst_id_di'],axis=1)#10124개
y=pd.read_csv('samp_train.csv').drop(['cst_id_di'],axis=1)#10124
y_nozero= y[(y['MRC_ID_DI']!=0)]#not 0!!!1924
#y_nozeroindex=y_nozero.loc[1:10119]
y_nozeroindexlist=y_nozero.index.tolist()
x_nozero=x.loc[y_nozeroindexlist]

In [3]:
print(x_nozero.shape)
print(y_nozero.shape)
print(x.shape)
print(y.shape)
y_nozeroarray=y_nozero.to_numpy()#1924
y_array=y.to_numpy()#10124
x_nozeroarray=x_nozero.to_numpy()#1924
x_array=x.to_numpy()#10124

(1924, 198)
(1924, 1)
(10124, 198)
(10124, 1)


In [5]:
x_train, x_test, y_train, y_test=x_nozeroarray[:1400], x_nozeroarray[1400:], y_nozeroarray[:1400], y_nozeroarray[1400:]
shuffle_index=np.random.permutation(1400)
x_train, y_train=x_train[shuffle_index], y_train[shuffle_index]

In [6]:
#Train the XGboost Model for Classification
import xgboost as xgb
model1 = xgb.XGBClassifier()
model2 = xgb.XGBClassifier(n_estimators=100, max_depth=8, learning_rate=0.1, subsample=0.5)

train_model1 = model1.fit(x_train, y_train)
train_model2 = model2.fit(x_train, y_train)


In [7]:
#prediction and Classification Report
from sklearn.metrics import classification_report

pred1 = train_model1.predict(x_test)
pred2 = train_model2.predict(x_test)

print('Model 1 XGboost Report %r' % (classification_report(y_test, pred1)))
print('Model 2 XGboost Report %r' % (classification_report(y_test, pred2)))

Model 1 XGboost Report '              precision    recall  f1-score   support\n\n           1       0.25      0.08      0.12        25\n           2       0.00      0.00      0.00         2\n           3       0.00      0.00      0.00        12\n           4       0.00      0.00      0.00         5\n           5       0.29      0.11      0.16        35\n           6       0.40      0.36      0.38       156\n           7       0.45      0.76      0.56       173\n           8       0.33      0.25      0.29        88\n           9       0.00      0.00      0.00        19\n          10       0.00      0.00      0.00         9\n\n    accuracy                           0.41       524\n   macro avg       0.17      0.16      0.15       524\nweighted avg       0.35      0.41      0.36       524\n'
Model 2 XGboost Report '              precision    recall  f1-score   support\n\n           1       0.50      0.04      0.07        25\n           2       0.00      0.00      0.00         2\n         

In [8]:
print("Accuracy for model 1: %.2f" % (accuracy_score(y_test, pred1) * 100))
print("Accuracy for model 2: %.2f" % (accuracy_score(y_test, pred2) * 100))

Accuracy for model 1: 41.03
Accuracy for model 2: 41.03


#0기준 이진분류, xgboost로

In [31]:
X_train, X_test, Y_train, Y_test=x_array[:8000], x_array[8000:], y_array[:8000], y_array[8000:]
shuffle_index=np.random.permutation(8000)
X_train, Y_train=X_train[shuffle_index], Y_train[shuffle_index]

Y_train_0=(Y_train==0)#0은 true, 0이 아니면 false
Y_test_0=(Y_test==0)



In [32]:
model1 = xgb.XGBClassifier()
model2 = xgb.XGBClassifier(n_estimators=100, max_depth=8, learning_rate=0.1, subsample=0.5)

train_model1 = model1.fit(X_train, Y_train_0)
train_model2 = model2.fit(X_train, Y_train_0)


In [33]:
pred1 = train_model1.predict(X_test)
pred2 = train_model2.predict(X_test)

print('Model 1 XGboost Report %r' % (classification_report(Y_test_0, pred1)))
print('Model 2 XGboost Report %r' % (classification_report(Y_test_0, pred2)))
print("Accuracy for model 1: %.2f" % (accuracy_score(Y_test_0, pred1) * 100))
print("Accuracy for model 2: %.2f" % (accuracy_score(Y_test_0, pred2) * 100))

Model 1 XGboost Report '              precision    recall  f1-score   support\n\n       False       0.63      0.37      0.47       382\n        True       0.87      0.95      0.91      1742\n\n    accuracy                           0.85      2124\n   macro avg       0.75      0.66      0.69      2124\nweighted avg       0.83      0.85      0.83      2124\n'
Model 2 XGboost Report '              precision    recall  f1-score   support\n\n       False       0.60      0.40      0.48       382\n        True       0.88      0.94      0.91      1742\n\n    accuracy                           0.84      2124\n   macro avg       0.74      0.67      0.69      2124\nweighted avg       0.83      0.84      0.83      2124\n'
Accuracy for model 1: 84.79
Accuracy for model 2: 84.37


8200/10124*100=80.99~81
, accuracy is higher than 81...괜찮지 않을까...라는 생각을...